In [1]:
from pyzx import *
from fractions import Fraction

In [2]:
class SQW:
    def __init__(self):
        self.qubit_amount = 3
        self.circ = Circuit(qubit_amount=self.qubit_amount)
    def setup(self,init=2):
        self.circ.add_gate("NOT",init)
    def add_layer(self, n=1):
        for i in range(n):
            self.circ.add_gate("XPhase",0 , phase=Fraction(2,3))
            self.circ.add_gate("TOF",0,1,2)
            self.circ.add_gate("CNOT",0,1)
            self.circ.add_gate("NOT",0)
            
            self.circ.add_gate("XPhase",0 , phase=Fraction(2,3))
            
            self.circ.add_gate("NOT",0)
            self.circ.add_gate("NOT",1)
            self.circ.add_gate("TOF",0,1,2)
            self.circ.add_gate("NOT",1)
            self.circ.add_gate("CNOT",0,1)

In [3]:
two_step = SQW()
two_step.setup()
two_step.add_layer(64)

In [4]:
g = two_step.circ
print("With syntactic sugar")
draw(g)
print("Without syntactic sugar")
draw(g.to_graph())
print(extract_simple(g.to_graph()).stats())

With syntactic sugar


Without syntactic sugar


Circuit  on 3 qubits with 2433 gates.
        1024 is the T-count
        1409 Cliffords among which
        896 2-qubit gates (896 CNOT, 0 other) and
        256 Hadamard gates.


# Hand picked simplifications
The simplifications below where chosen because of the structure of the diagram above.

- phase_free_simp -> spider_simp followed by a bialg_simp
- id_simp to remove any $n*2\pi$ rotations
- spider_simp to fuse spiders after the id
- id_simp to remove any $n*2\pi$ rotations

In [5]:
def qw_opt(g):
    phase_free_simp(g,quiet=False)
    id_simp(g,quiet=False)
    spider_simp(g, quiet=False)
    id_simp(g,quiet=False)
    #supplementarity_simp(g,quiet=False)
    g.normalize()

In [6]:
opt = g.to_graph()
qw_opt(opt)
draw(opt)

spider_simp: 466. 325. 228. 139. 107. 15.  6 iterations
id_simp: 129. 127.  2 iterations


In [7]:
result = extract_simple(opt)
print(result.stats(),"\nEQUAL: " , g.verify_equality(result))
print("Result:")
draw(result)
print("Original:")
draw(g.to_graph())
print(extract_simple(g.to_graph()).stats())

Circuit  on 3 qubits with 1795 gates.
        1024 is the T-count
        771 Cliffords among which
        640 2-qubit gates (640 CNOT, 0 other) and
        2 Hadamard gates. 
EQUAL:  True
Result:


Original:


Circuit  on 3 qubits with 2433 gates.
        1024 is the T-count
        1409 Cliffords among which
        896 2-qubit gates (896 CNOT, 0 other) and
        256 Hadamard gates.


# Full Reduce
This simplification is useful to reduce the T-count of a circuit due to the phase gadget optimizations implemented in the algorithm

In [8]:
fr = g.to_graph()
full_reduce(fr, quiet=False)
fr.normalize()
draw(fr)

spider_simp: 466. 325. 228. 139. 107. 15.  6 iterations
id_simp: 129. 127.  2 iterations
pivot_simp: 1.  1 iterations
pivot_gadget_simp: 403. 152. 101. 53. 32. 12. 6. 4. 2. 1.  10 iterations
id_simp: 128.  1 iterations
spider_simp: 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.  127 iterations
gadget_simp: 324.  1 iterations
id_simp: 72.  1 iterations
spider_simp: 72.  1 iterations
lcomp_simp: 254. 4. 4. 10. 6. 6. 4. 10. 4. 4. 14. 8. 6. 4. 6. 16. 4. 6. 14. 6. 4. 6. 6. 4. 6. 8. 8. 8. 8. 8. 4. 10. 8. 6. 4. 10. 6. 4.  38 iterations
id_simp: 1.  1 iterations


In [9]:
fr_opt = extract_circuit(fr.copy())
print(fr_opt.stats(),"\nEQUAL: " , g.verify_equality(fr_opt))
draw(fr_opt)

Circuit  on 3 qubits with 1300 gates.
        388 is the T-count
        912 Cliffords among which
        389 2-qubit gates (250 CNOT, 139 other) and
        522 Hadamard gates. 
EQUAL:  True


PyZX tends to perform better with a lot of hadamard gates, although this circuit could use some optimizations

We can remove some hadamards by some colour changes and the total number of gates with a few spider simps and id simps

In [10]:
fr_graph = fr_opt.to_graph()
# Simple simplifications
spider_simp(fr_graph,quiet=False)
id_simp(fr_graph,quiet=False)
to_rg(fr_graph)
#The simps above dont change the graph too much so we can use the extract_simple
fr_opt = extract_simple(fr_graph)
print()
print(fr_opt.stats(),"\nEQUAL: " , g.verify_equality(fr_opt))
draw(fr_opt)
draw(fr)

spider_simp: 349. 261. 195. 131. 81. 39. 29. 26. 13. 13. 8. 6. 3. 3. 2. 1. 1.  17 iterations
id_simp: 2.  1 iterations

Circuit  on 3 qubits with 786 gates.
        388 is the T-count
        398 Cliffords among which
        391 2-qubit gates (197 CNOT, 194 other) and
        6 Hadamard gates. 
EQUAL:  True


In [11]:
#settings.tikzit_location = "/usr/bin/tikzit"
#tikz.tikzit(fr_opt)

In [12]:
indexes = list(filter(lambda x: isinstance(fr_opt.to_graph().phases()[x],Fraction),fr_opt.to_graph().phases())) 

In [13]:
rotations = list(map(lambda x: fr_opt.to_graph().phases()[x], indexes))

In [14]:
useful = rotations[4:]

for i in range(len(useful)):
    print(str(useful[i]) + " " , end="")
    if (i+1) % 6 == 0:
        print()


len(rotations)//6, len(rotations)

1/4 2/3 1/4 7/4 5/3 1/4 
7/4 5/3 7/4 1/4 5/3 1/4 
7/4 5/3 7/4 1/4 2/3 7/4 
1/4 2/3 7/4 1/4 2/3 1/4 
7/4 5/3 7/4 1/4 2/3 1/4 
7/4 5/3 1/4 7/4 2/3 7/4 
1/4 2/3 1/4 7/4 2/3 7/4 
1/4 2/3 7/4 1/4 2/3 7/4 
1/4 2/3 1/4 7/4 5/3 1/4 
7/4 5/3 7/4 1/4 2/3 1/4 
7/4 2/3 7/4 1/4 5/3 1/4 
7/4 5/3 1/4 7/4 5/3 1/4 
7/4 2/3 1/4 7/4 5/3 1/4 
7/4 2/3 1/4 7/4 2/3 1/4 
7/4 2/3 7/4 1/4 2/3 7/4 
1/4 5/3 7/4 1/4 2/3 1/4 
7/4 5/3 7/4 1/4 2/3 1/4 
7/4 2/3 1/4 7/4 2/3 1/4 
7/4 5/3 1/4 7/4 5/3 7/4 
1/4 2/3 7/4 1/4 2/3 7/4 
1/4 2/3 1/4 7/4 2/3 7/4 
1/4 2/3 1/4 7/4 2/3 7/4 
1/4 2/3 1/4 7/4 2/3 1/4 
7/4 2/3 1/4 7/4 2/3 1/4 
7/4 5/3 1/4 7/4 5/3 7/4 
1/4 2/3 7/4 1/4 2/3 1/4 
7/4 2/3 7/4 1/4 5/3 1/4 
7/4 5/3 7/4 1/4 5/3 7/4 
1/4 2/3 1/4 7/4 2/3 1/4 
7/4 5/3 7/4 1/4 2/3 7/4 
1/4 2/3 7/4 1/4 5/3 7/4 
1/4 5/3 7/4 1/4 2/3 1/4 
7/4 2/3 7/4 1/4 2/3 7/4 
1/4 5/3 1/4 7/4 5/3 7/4 
1/4 5/3 1/4 7/4 5/3 7/4 
1/4 2/3 7/4 1/4 5/3 7/4 
1/4 5/3 7/4 1/4 2/3 7/4 
1/4 2/3 7/4 1/4 5/3 1/4 
7/4 2/3 7/4 1/4 5/3 7/4 
1/4 2/3 7/4 1/4 2/3 1/4 


(64, 389)